# Temperatura y humedad relativa

El objetivo de este notebook es crear una tabla con los valores medios de cada mes de temperatura y humedad relativa para incluirlos como parámetros de NoiseModelling en el bloque `Noise_level_from_source`, el cual genera la capa de receptores con los niveles de ruido.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dbfread import DBF
import geopandas as gpd
import datetime
import glob
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Variables
meteo = pd.read_csv("../Data_input/Meteorologia/meteo22.csv", sep=";")
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
columnas_val = ['V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10',
                'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
                'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31']
columnas_dias = ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10',
                 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20',
                 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31']


In [3]:
def medidas_meteo(meteo_df, magnitud, mes, dias, vals):
    """
    Calcula la el valor medio de todas las estaciones meteorológicas
    de la magnitud y el mes dado.
    
    Parámetros de entrada:
    - meteo_df: dataset con los datos de las estaciones meteorológicas
    - magnitud: código de la magnitud
    - mes: mes (entero)
    - dias: lista de columnas de días del dataset inicial
    - vals: lista de columnas de la validación de los valores del dataset inicial
    
    Devuelve:
    - Valor medio de la magnitud.
    """
    meteo_interes = meteo.loc[(meteo['MAGNITUD']==magnitud) & 
                              (meteo['MES']==mes)]
    
    # Mediciones del parámetro
    columnas_dias = meteo_interes[dias].reset_index(drop=True)
    lista_dias = columnas_dias.to_numpy().tolist()
    lista_dias = list(itertools.chain(*lista_dias))
    
    # Validaciones del parámetro
    columnas_val = meteo_interes[vals].reset_index(drop=True)
    lista_val = columnas_val.to_numpy().tolist()
    lista_val = list(itertools.chain(*lista_val))
    
    # Lista de mediciones válidas
    lista_valida = []
    for temp, val in zip(lista_dias, lista_val):
        if val=='V':
            lista_valida.append(temp)
    
    suma = sum(lista_valida)
    registros = len(lista_valida)
    media_parametro = round(suma/registros, 1)
    return media_parametro

In [4]:
def dos_magnitudes_meses(meses, nom_magn1, cod_magn1, nom_magn2, cod_magn2):
    """
    Calcula el valor de las dos magnitudes para los meses dados.
    
    Parámetros de entrada:
    - meeses: lista de meses en número
    - nom_magn1: nombre de la magnitud 1
    - cod_magn1: código de la magnitud 1
    - nom_magn2: nombre de la magnitud 2
    - cod_magn2: código de la magnitud 2
    
    Devuelve:
    - Tabla que contiene los valores de las magnitudes (columnas) para
    los meses (filas)
    """
    # Se crea un nuevo dataframe con los dos parámetros
    temp_hr_df = pd.DataFrame()
    temp_hr_df[nom_magn1] = None
    temp_hr_df[nom_magn2] = None
    
    lista_temp = []
    lista_hr = []
    
    for mes in meses:
        # Se añade la magnitud 1 (temperatura)
        temp = medidas_meteo(meteo, cod_magn1, mes, columnas_dias, columnas_val)
        lista_temp.append(temp)
        # Se añade la magnitud 2 (humedad relativa)
        hr = medidas_meteo(meteo, cod_magn2, mes, columnas_dias, columnas_val)
        lista_hr.append(hr)
        
    temp_hr_df[nom_magn1] = lista_temp
    temp_hr_df[nom_magn2] = lista_hr

    return temp_hr_df

In [5]:
dos_magnitudes_meses(meses, 'Temperatura', 83, 'Humedad Relativa', 86)

,Temperatura,Humedad Relativa
0,6.0,61.3
1,9.6,55.9
2,10.0,69.2
3,12.5,58.5
4,20.7,43.2
5,24.8,35.0
6,29.5,29.7
7,27.8,34.3
8,21.8,49.6
9,18.6,61.4
